<a href="https://colab.research.google.com/github/ArshT/Mini-Projects/blob/master/Cifar_10_CNN_MobileNet_Tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###Importing the Important Libraries

import keras
from keras.layers.core import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D,Input
from keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [0]:
###Importing a Data

from keras.datasets.cifar10 import load_data

(X_train_orig,Y_train_orig),(X_test_orig,Y_test_orig) = load_data()

In [3]:
###Overview of the Imported Data

print('X_train Shape:',X_train_orig.shape)
print('Y_train Shape:',Y_train_orig.shape)
print('X_test Shape:',X_test_orig.shape)
print('Y_test Shape:',Y_test_orig.shape)

X_train Shape: (50000, 32, 32, 3)
Y_train Shape: (50000, 1)
X_test Shape: (10000, 32, 32, 3)
Y_test Shape: (10000, 1)


In [0]:
###Pre-Processing Data

##Normalizing the Data
X_train = X_train_orig / 255
X_test = X_test_orig /255

In [0]:
##One hot Encoding

Y_train = keras.utils.to_categorical(Y_train_orig,10)
Y_test = keras.utils.to_categorical(Y_test_orig,10)

In [6]:
###Overview of Processed Data


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


In [7]:
###Creating a Model
input_shape = (32,32,3)

model = Sequential()

#1 and 2
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='SAME'))
model.add(Dropout(0.3))


#3 and 4
model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='SAME'))
model.add(Dropout(0.3))


#5 and 6
model.add(Conv2D(128,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='SAME'))
model.add(Dropout(0.3))


#7 and 8
model.add(Conv2D(256,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='SAME'))
model.add(Dropout(0.3))


#9(Flat)
model.add(Flatten())

#10 and 11
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

#12 and 13
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))

#14
model.add(Dense(10,activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
###Compiling the Model

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])

In [9]:
###Training the Model

history = model.fit(X_train,Y_train,batch_size=64,epochs=25,verbose=1,validation_data=(X_test,Y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples
Epoch 1/25





50000/50000 [==============================] - 12s 233us/step - loss: 1.7889 - categorical_accuracy: 0.3265 - val_loss: 1.4818 - val_categorical_accuracy: 0.4542
Epoch 2/25
50000/50000 [==============================] - 9s 179us/step - loss: 1.4030 - categorical_accuracy: 0.4907 - val_loss: 1.1947 - val_categorical_accuracy: 0.5710
Epoch 3/25
50000/50000 [==============================] - 9s 181us/step - loss: 1.2550 - categorical_accuracy: 0.5507 - val_loss: 1.1632 - val_categorical_accuracy: 0.5903
Epoch 4/25
50000/50000 [==============================] - 9s 183us/step - loss: 1.1505 - categorical_accuracy: 0.5912 - val_loss: 0.9614 - val_categorical_accuracy: 0.6626
Epoch 5/25
50000/50000 [==============================] - 9s 183us/step - loss: 1.0903 - categorical_accuracy: 0.6154 - val_loss: 0.9667 - val_categorical_accu

In [0]:
####Transfer Learning using MobileNet

In [0]:
###Resizing the Image

from skimage.transform import resize

target_size = 96

def procces_image(x):
  x = resize(x,(target_size,target_size),mode='constant',anti_aliasing=False)

  return x.astype(np.float32)

In [0]:
###Making the a generator for iterating through all the images

from sklearn.utils import shuffle

def load_data_generator(x, y, batch_size=64):
    num_samples = x.shape[0]
    while 1:  # Loop forever so the generator never terminates
        try:
            shuffle(x)
            for i in range(0, num_samples, batch_size):
                x_data = [procces_image(im) for im in x[i:i+batch_size]]
                y_data = y[i:i + batch_size]
            
                # convert to numpy array since this what keras required
                yield shuffle(np.array(x_data), np.array(y_data))
        except Exception as err:
            print(err)

In [14]:
###Making the Model

from keras.applications import MobileNet
from keras.models import Model

#Input Tensor
input_tensor = Input(shape=(target_size,target_size,3))

#Taking the base model as MobileNet
base_model = MobileNet(include_top=False,
                        weights='imagenet',
                       input_tensor=input_tensor,
                       input_shape=(target_size,target_size,3),
                       pooling='avg')

#Unfreezing all Layers
for layer in base_model.layers:
  layer.trainable=True

#Adding my own Layers
op1 = Dense(256,activation='relu')(base_model.output)
op1 = Dropout(0.3)(op1)
op2 = Dense(256,activation='relu')(op1)
op2 = Dropout(0.3)(op2)

#Final Layer
output_tensor = Dense(10, activation='softmax')(op2)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
###Assigning the Model

model = Model(inputs=input_tensor, outputs=output_tensor)

In [0]:
###Compiling the Model

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [17]:
###Training the Model
import math

batch_size=128
train_generator = load_data_generator(X_train, Y_train, batch_size)
m=X_train.shape[0]

model.fit_generator(
    generator=train_generator,
    steps_per_epoch=math.ceil(m/batch_size),
    verbose=1,
    epochs=5)

Epoch 1/5
391/391 [==============================] - 141s 361ms/step - loss: 0.6065 - categorical_accuracy: 0.8098
Epoch 2/5
391/391 [==============================] - 136s 347ms/step - loss: 0.3160 - categorical_accuracy: 0.8996
Epoch 3/5
391/391 [==============================] - 136s 347ms/step - loss: 0.2316 - categorical_accuracy: 0.9261
Epoch 4/5
391/391 [==============================] - 137s 351ms/step - loss: 0.1840 - categorical_accuracy: 0.9427
Epoch 5/5
391/391 [==============================] - 136s 348ms/step - loss: 0.1551 - categorical_accuracy: 0.9506


In [18]:
test_generator = load_data_generator(X_test, Y_test, batch_size=128)
m_test = X_test.shape[0]
model.evaluate_generator(generator=test_generator,
                         steps=math.ceil(m_test/batch_size),
                         verbose=1)

79/79 [==============================] - 23s 296ms/step


[0.48796472742557523, 0.863]